In [1]:
!pip install -q shap
import shutil
import os
import sys
import base64
import pickle
import shap
import numpy as np
from os.path import exists, join, basename, splitext
import time
import matplotlib
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.pyplot as pyplt

basepath_drive='/notebooks/Spectra/'
os.chdir(join(basepath_drive,'Analisys'))
from utils import *
test_path=join(basepath_drive,'Esperimenti')
samples_path=join(test_path,'samples')
processed_path=join(test_path,'processed')
processed_3d_path=join(test_path,'processed_3d')
shap_path=join(test_path,'shap/shap_est_1_n_samples_2450_n_evals_10000.pkl')
with open(shap_path, 'rb') as file:
   shap_values  = pickle.load(file)
shap_values.values = [val for val in np.moveaxis(shap_values.values[0], -1, 0)]
shap_values.values = [ v[None,...] for v in shap_values.values ]
top1=[shap_values.values[0]]
motion=np.load(join(processed_3d_path,'es1_t_3D.npy'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 7.1 MB/s eta 0:00:0000:0100:01
Mounted at /content/drive


In [ ]:
index=np.array([80 ,100,120,140 ])
fig_size=20
delta=0.1
mono=True
video_path_2d = join(processed_path,'es1_t_processed.mp4')
extract_2d_frames(motion,top1[0][0],index,video_path_2d,fig_size=fig_size,delta=delta,mono=mono)
video_path_3d = join(processed_path,'es1_t_processed.mp4')
extract_3d_frames(motion,top1[0][0],index,video_path_3d,azim=45,fig_size=fig_size,delta=2*delta,mono=mono)

In [ ]:
class_names=get_action_names()

reduced_channel = top1[0][0].sum(-1)
index=top_n_indices(reduced_channel,3)
index.sort()

shap.image_plot(
    shap_values=[np.transpose(top1[0], (0,2,1,3))],
    pixel_values=np.transpose(((shap_values.data+1)/2),(0,2,1,3)),
    labels=shap_values.output_names,
    true_labels=[class_names[41]],
    show=False)

fig = pyplt.gcf()

_, h = fig.get_size_inches()
fig.set_size_inches(2*h, h)
h = fig.get_figheight()
fig.set_figheight(2*h)
h = fig.get_figwidth()
fig.set_figwidth(2*h)

axs=fig.get_axes()
pyplt.delaxes(axs[0])
axs[1].set_aspect(5)
axs[1].set_position([0.16, 0.3, 0.7, 1])
axs[1].axis('on')
axs[1].spines[:].set_visible(False)
axs[1].yaxis.set_major_locator(MultipleLocator(1))
axs[1].set_yticklabels([H36_map[i] for i in range(17)])
axs[1].set_position([0.16, 0.3, 0.7, 1])
class_pred=axs[1].get_title()
axs[1].set_title(f'\nPredicted class: {class_pred}\n',fontsize=36)
pyplt.show()

video_path = join(processed_path,'es1_t_processed.mp4')
apply_shap_on_3d(motion,top1[0][0],index,video_path,azim=80,fig_size=20)